Importing the required Modules

In [50]:
import random
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.optimizers import SGD
import numpy as np
import pickle
import json
import nltk
from nltk.stem import WordNetLemmatizer
import pandas as pd


In [51]:
lemmatizer = WordNetLemmatizer()

Download the nltk packages

In [52]:
nltk.download('omw-1.4')
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\prath\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\prath\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\prath\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Init File of Intents.json

In [53]:
words = []
classes = []
documents = []
ignore_words = ["?", "!",","]

read the intents.json file

In [54]:
data_file = open("../intents.json").read()
data_file

'{\n  "intent": [\n    {\n      "tag": "greeting",\n      "patterns": [\n        "Hi",\n        "How are you?",\n        "Hey there, how have you been?",\n        "Is anyone there?",\n        "Hello",\n        "Good day",\n        "Hiii",\n        "Hii",\n        "AyurBot"\n      ],\n      "responses": [\n        "Hi there, what can I do for you?",\n        "Hey there, how can I help?"\n      ]\n    },\n    {\n      "tag": "goodbye",\n      "patterns": ["Bye", "See you there", "Goodbye"],\n      "responses": [\n        "See you later, thanks for visiting ",\n        "Have a nice day",\n        "Bye! Come back again soon."\n      ]\n    },\n    {\n      "tag": "thanks",\n      "patterns": [\n        "Thanks",\n        "Thank you",\n        "That\'s helpful",\n        "Nice",\n        "Good",\n        "Thank you Bot",\n        "Thank you soo much",\n        "You are Beautiful Bot",\n        "You are Great"\n      ],\n      "responses": ["Happy to help!", "Any time!", "My pleasure"]\n    

Reading the Json file from Jsilo online Storage of JSON

In [55]:
import requests

In [56]:
url = 'https://api.jsonsilo.com/5ff310a1-b136-45a7-86c3-0b7378ebfd6c'
headers = {
        'X-SILO-KEY': 'pVyv9PBnSNfIhrqS6piqF2P2npNqMydLp0kNxp8igK',
        'Content-Type': 'application/json'
      }

In [57]:
new_data=requests.get(url,headers=headers)

In [58]:
intents=new_data.json()
intents

{'intent': [{'tag': 'greeting',
   'patterns': ['Hi',
    'How are you?',
    'Hey there, how have you been?',
    'Is anyone there?',
    'Hello',
    'Good day',
    'Hiii',
    'Hii',
    'AyurBot'],
   'responses': ['Hi there, what can I do for you?',
    'Hey there, how can I help?']},
  {'tag': 'src_code',
   'patterns': ['What is your Source Code',
    'Source Code',
    'Where is your Source Code'],
   'responses': ['My Source Code is Available in Github Repository https://github.com/PrathameshDhande22/Prakriti-Determine.']},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you there', 'Goodbye'],
   'responses': ['See you later, thanks for visiting ',
    'Have a nice day',
    'Bye! Come back again soon.']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Nice',
    'Good',
    'Thank you Bot',
    'Thank you soo much',
    'You are Beautiful Bot',
    'You are Great'],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure']},


Converting the intents file from bytes to json 

In [ ]:
intents=json.loads(data_file)
intents

In [59]:
for intent in intents["intent"]:
    for pattern in intent["patterns"]:
        # separate each word from the pattern
        w=nltk.word_tokenize(pattern)
        # add it in the words list
        words.extend(w)
        
        # adding it in the documents with the defined tag.
        documents.append((w,intent["tag"]))

        # adding the classes in the classes list
        if intent["tag"] not in classes:
            classes.append(intent["tag"])

In [60]:
words

['Hi',
 'How',
 'are',
 'you',
 '?',
 'Hey',
 'there',
 ',',
 'how',
 'have',
 'you',
 'been',
 '?',
 'Is',
 'anyone',
 'there',
 '?',
 'Hello',
 'Good',
 'day',
 'Hiii',
 'Hii',
 'AyurBot',
 'What',
 'is',
 'your',
 'Source',
 'Code',
 'Source',
 'Code',
 'Where',
 'is',
 'your',
 'Source',
 'Code',
 'Bye',
 'See',
 'you',
 'there',
 'Goodbye',
 'Thanks',
 'Thank',
 'you',
 'That',
 "'s",
 'helpful',
 'Nice',
 'Good',
 'Thank',
 'you',
 'Bot',
 'Thank',
 'you',
 'soo',
 'much',
 'You',
 'are',
 'Beautiful',
 'Bot',
 'You',
 'are',
 'Great',
 'about',
 'you',
 'Who',
 'are',
 'you',
 '?',
 'What',
 'are',
 'you',
 '?',
 'Who',
 'you',
 'are',
 '?',
 'what',
 'you',
 'can',
 'do',
 'What',
 'is',
 'the',
 'Purpose',
 'of',
 'the',
 'project',
 'what',
 'is',
 'your',
 'name',
 'what',
 'should',
 'I',
 'call',
 'you',
 'whats',
 'your',
 'name',
 '?',
 'Could',
 'you',
 'help',
 'me',
 '?',
 'give',
 'me',
 'a',
 'hand',
 'please',
 'Can',
 'you',
 'help',
 '?',
 'What',
 'can',
 'you',

In [61]:
for doc in documents:
    print(doc)

(['Hi'], 'greeting')
(['How', 'are', 'you', '?'], 'greeting')
(['Hey', 'there', ',', 'how', 'have', 'you', 'been', '?'], 'greeting')
(['Is', 'anyone', 'there', '?'], 'greeting')
(['Hello'], 'greeting')
(['Good', 'day'], 'greeting')
(['Hiii'], 'greeting')
(['Hii'], 'greeting')
(['AyurBot'], 'greeting')
(['What', 'is', 'your', 'Source', 'Code'], 'src_code')
(['Source', 'Code'], 'src_code')
(['Where', 'is', 'your', 'Source', 'Code'], 'src_code')
(['Bye'], 'goodbye')
(['See', 'you', 'there'], 'goodbye')
(['Goodbye'], 'goodbye')
(['Thanks'], 'thanks')
(['Thank', 'you'], 'thanks')
(['That', "'s", 'helpful'], 'thanks')
(['Nice'], 'thanks')
(['Good'], 'thanks')
(['Thank', 'you', 'Bot'], 'thanks')
(['Thank', 'you', 'soo', 'much'], 'thanks')
(['You', 'are', 'Beautiful', 'Bot'], 'thanks')
(['You', 'are', 'Great'], 'thanks')
(['about', 'you'], 'about')
(['Who', 'are', 'you', '?'], 'about')
(['What', 'are', 'you', '?'], 'about')
(['Who', 'you', 'are', '?'], 'about')
(['what', 'you', 'can', 'do'], '

In [62]:
classes

['greeting',
 'src_code',
 'goodbye',
 'thanks',
 'about',
 'name',
 'help',
 'noanswer',
 'prakriti',
 'creators',
 'badwords',
 'acknowledge',
 'age_inquiry',
 'positive_feedback',
 'negative_feedback',
 'information_request',
 'contact_support',
 'diet_recommendation',
 'exercise_suggestions',
 'stress_management',
 'sleep_tips',
 'nutrition_information',
 'exit',
 'info_prakriti',
 'info_dosha',
 'info_kapha',
 'info_vata',
 'info_pitta']

Lemmatization on words and removing the words which are not usable

In [63]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words

['hi',
 'how',
 'are',
 'you',
 'hey',
 'there',
 'how',
 'have',
 'you',
 'been',
 'is',
 'anyone',
 'there',
 'hello',
 'good',
 'day',
 'hiii',
 'hii',
 'ayurbot',
 'what',
 'is',
 'your',
 'source',
 'code',
 'source',
 'code',
 'where',
 'is',
 'your',
 'source',
 'code',
 'bye',
 'see',
 'you',
 'there',
 'goodbye',
 'thanks',
 'thank',
 'you',
 'that',
 "'s",
 'helpful',
 'nice',
 'good',
 'thank',
 'you',
 'bot',
 'thank',
 'you',
 'soo',
 'much',
 'you',
 'are',
 'beautiful',
 'bot',
 'you',
 'are',
 'great',
 'about',
 'you',
 'who',
 'are',
 'you',
 'what',
 'are',
 'you',
 'who',
 'you',
 'are',
 'what',
 'you',
 'can',
 'do',
 'what',
 'is',
 'the',
 'purpose',
 'of',
 'the',
 'project',
 'what',
 'is',
 'your',
 'name',
 'what',
 'should',
 'i',
 'call',
 'you',
 'whats',
 'your',
 'name',
 'could',
 'you',
 'help',
 'me',
 'give',
 'me',
 'a',
 'hand',
 'please',
 'can',
 'you',
 'help',
 'what',
 'can',
 'you',
 'do',
 'for',
 'me',
 'i',
 'need',
 'a',
 'support',
 'i'

Sorting the words

In [64]:
words = sorted(list(set(words)))
words

["'m",
 "'re",
 "'s",
 '.',
 'a',
 'about',
 'age',
 'annoying',
 'anyone',
 'aobut',
 'are',
 'away',
 'ayurbot',
 'bad',
 'be',
 'beautiful',
 'been',
 'believe',
 'best',
 'bot',
 'by',
 'bye',
 'ca',
 'call',
 'can',
 'chatbot',
 'chatgpt',
 'code',
 'complete',
 'contact',
 'could',
 'created',
 'creator',
 'customer',
 'day',
 'despise',
 'detail',
 'determine',
 'diet',
 'disappointed',
 'do',
 'doe',
 'doing',
 'doshas',
 'eat',
 'eating',
 'exercise',
 'exit',
 'explain',
 'fact',
 'feeling',
 'figure',
 'find',
 'food',
 'for',
 'fuck',
 'get',
 'give',
 'go',
 'good',
 'goodbye',
 'great',
 'guide',
 'hand',
 'hate',
 'have',
 'healthy',
 'hello',
 'help',
 'helpful',
 'hey',
 'hi',
 'hii',
 'hiii',
 'how',
 'i',
 'identify',
 'impressed',
 'improve',
 'in',
 'information',
 'irrelevant',
 'is',
 'job',
 'kapha',
 'learn',
 'love',
 'made',
 'make',
 'manage',
 'me',
 'mean',
 'more',
 'much',
 'my',
 "n't",
 'name',
 'need',
 'nice',
 'not',
 'nothing',
 'noting',
 'now',
 

Sorting the Classes

In [65]:
classes = sorted(list(set(classes)))
classes

['about',
 'acknowledge',
 'age_inquiry',
 'badwords',
 'contact_support',
 'creators',
 'diet_recommendation',
 'exercise_suggestions',
 'exit',
 'goodbye',
 'greeting',
 'help',
 'info_dosha',
 'info_kapha',
 'info_pitta',
 'info_prakriti',
 'info_vata',
 'information_request',
 'name',
 'negative_feedback',
 'noanswer',
 'nutrition_information',
 'positive_feedback',
 'prakriti',
 'sleep_tips',
 'src_code',
 'stress_management',
 'thanks']

Classes and words and Documents overview

In [66]:
print(len(documents), "documents")

print(len(classes), "classes", classes)

print(len(words), "unique lemmatized words", words)

135 documents
28 classes ['about', 'acknowledge', 'age_inquiry', 'badwords', 'contact_support', 'creators', 'diet_recommendation', 'exercise_suggestions', 'exit', 'goodbye', 'greeting', 'help', 'info_dosha', 'info_kapha', 'info_pitta', 'info_prakriti', 'info_vata', 'information_request', 'name', 'negative_feedback', 'noanswer', 'nutrition_information', 'positive_feedback', 'prakriti', 'sleep_tips', 'src_code', 'stress_management', 'thanks']
170 unique lemmatized words ["'m", "'re", "'s", '.', 'a', 'about', 'age', 'annoying', 'anyone', 'aobut', 'are', 'away', 'ayurbot', 'bad', 'be', 'beautiful', 'been', 'believe', 'best', 'bot', 'by', 'bye', 'ca', 'call', 'can', 'chatbot', 'chatgpt', 'code', 'complete', 'contact', 'could', 'created', 'creator', 'customer', 'day', 'despise', 'detail', 'determine', 'diet', 'disappointed', 'do', 'doe', 'doing', 'doshas', 'eat', 'eating', 'exercise', 'exit', 'explain', 'fact', 'feeling', 'figure', 'find', 'food', 'for', 'fuck', 'get', 'give', 'go', 'good', 

In [67]:
documents

[(['Hi'], 'greeting'),
 (['How', 'are', 'you', '?'], 'greeting'),
 (['Hey', 'there', ',', 'how', 'have', 'you', 'been', '?'], 'greeting'),
 (['Is', 'anyone', 'there', '?'], 'greeting'),
 (['Hello'], 'greeting'),
 (['Good', 'day'], 'greeting'),
 (['Hiii'], 'greeting'),
 (['Hii'], 'greeting'),
 (['AyurBot'], 'greeting'),
 (['What', 'is', 'your', 'Source', 'Code'], 'src_code'),
 (['Source', 'Code'], 'src_code'),
 (['Where', 'is', 'your', 'Source', 'Code'], 'src_code'),
 (['Bye'], 'goodbye'),
 (['See', 'you', 'there'], 'goodbye'),
 (['Goodbye'], 'goodbye'),
 (['Thanks'], 'thanks'),
 (['Thank', 'you'], 'thanks'),
 (['That', "'s", 'helpful'], 'thanks'),
 (['Nice'], 'thanks'),
 (['Good'], 'thanks'),
 (['Thank', 'you', 'Bot'], 'thanks'),
 (['Thank', 'you', 'soo', 'much'], 'thanks'),
 (['You', 'are', 'Beautiful', 'Bot'], 'thanks'),
 (['You', 'are', 'Great'], 'thanks'),
 (['about', 'you'], 'about'),
 (['Who', 'are', 'you', '?'], 'about'),
 (['What', 'are', 'you', '?'], 'about'),
 (['Who', 'you',

dumping the words list and classes list

In [68]:
# pickle.dump(words, open("words.pkl", "wb"))
# pickle.dump(classes, open("classes.pkl", "wb"))

Training initializer

In [69]:
training=[]

In [70]:
output_empty=[0]*len(classes)
output_empty

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [71]:
for doc in documents:
    bag=[]
    # extracting the patterns
    pattern_words=doc[0]
    
    # lemmatizing the words from the pattern
    pattern_words=[lemmatizer.lemmatize(word.lower()) for word in pattern_words]

    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
     # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row=list(output_empty)
    output_row[classes.index(doc[1])]=1
    
    training.append([bag,output_row])
    
print(training)

[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

Shuffling our features and turn into np.array

In [72]:
# random.shuffle(training)
print(training[0][0])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [73]:
print(training[0])

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [74]:
for train in training:
    print(train)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [75]:
training_data=np.array(training,dtype=object)
training_data

array([[list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
        list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])],
       [list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [76]:
training_data.shape

(135, 2)

Training X and Training Y data

In [77]:
train_x=list(training_data[:,0])
train_x

[[0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,

In [78]:
train_y=list(training_data[:,1])
train_y

[[0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0

Creating the Model

In [79]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation="softmax"))

Summary of the model

In [80]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 128)               21888     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 28)                1820      
                                                                 
Total params: 31964 (124.86 KB)
Trainable params: 31964 (124.86 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model

In [81]:
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=["accuracy"])

In [82]:
model.fit(np.array(train_x),np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
27/27 [==============================] - 1s 2ms/step - loss: 3.3506 - accuracy: 0.0222
Epoch 2/200
27/27 [==============================] - 0s 5ms/step - loss: 3.3187 - accuracy: 0.0444
Epoch 3/200
27/27 [==============================] - 0s 2ms/step - loss: 3.2382 - accuracy: 0.0741
Epoch 4/200
27/27 [==============================] - 0s 3ms/step - loss: 3.1865 - accuracy: 0.1259
Epoch 5/200
27/27 [==============================] - 0s 5ms/step - loss: 3.1454 - accuracy: 0.1481
Epoch 6/200
27/27 [==============================] - 0s 2ms/step - loss: 3.0533 - accuracy: 0.1259
Epoch 7/200
27/27 [==============================] - 0s 2ms/step - loss: 2.9498 - accuracy: 0.1704
Epoch 8/200
27/27 [==============================] - 0s 5ms/step - loss: 2.9075 - accuracy: 0.2074
Epoch 9/200
27/27 [==============================] - 0s 2ms/step - loss: 2.8456 - accuracy: 0.1926
Epoch 10/200
27/27 [==============================] - 0s 4ms/step - loss: 2.7454 - accuracy: 0.2370
Epoch 11/

In [83]:
np.array(train_x)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

The Model is Trained

Lets predict it using the model

In [84]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

In [85]:
clean_up_sentence("hello who are you")

['hello', 'who', 'are', 'you']

In [86]:
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w)
    return np.array(bag)


In [87]:
bow("hello who are you",words)

found in bag: hello
found in bag: who
found in bag: are
found in bag: you


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0])

In [88]:
words[10]

'are'

In [89]:
def predict_class(sentence, model):
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    print(res)
    index=np.argmax(res)
    probmax=np.max(res)
    print("maximum index =",index,"probabiliy = ",probmax)
    results={"intent": classes[index], "probability": str(probmax)}
    return results


In [91]:
predict_class("source code",model)

1/1 [==============================] - 0s 61ms/step


[1.1574853e-14 9.5326369e-09 4.3513040e-08 7.0933792e-14 2.1599963e-06
 3.8922358e-11 4.9235338e-14 2.0948534e-09 3.5325613e-08 1.2096518e-10
 6.6025324e-10 7.1856685e-09 2.5194644e-09 2.3273753e-08 7.3510509e-09
 1.0438244e-09 7.4113096e-08 3.2386416e-10 2.3401985e-09 5.3935346e-12
 2.4149907e-13 9.7792198e-11 3.9399047e-11 3.0961744e-10 1.1912894e-09
 9.9999762e-01 3.9610359e-09 2.6771334e-15]
maximum index = 25 probabiliy =  0.9999976


{'intent': 'src_code', 'probability': '0.9999976'}

In [92]:
classes

['about',
 'acknowledge',
 'age_inquiry',
 'badwords',
 'contact_support',
 'creators',
 'diet_recommendation',
 'exercise_suggestions',
 'exit',
 'goodbye',
 'greeting',
 'help',
 'info_dosha',
 'info_kapha',
 'info_pitta',
 'info_prakriti',
 'info_vata',
 'information_request',
 'name',
 'negative_feedback',
 'noanswer',
 'nutrition_information',
 'positive_feedback',
 'prakriti',
 'sleep_tips',
 'src_code',
 'stress_management',
 'thanks']

In [93]:
def getResponse(ints, intents_json):
    tag = ints["intent"]
    list_of_intents = intents_json["intent"]
    for i in list_of_intents:
        if i["tag"] == tag:
            result = random.choice(i["responses"])
            break
    return result

In [94]:
getResponse(predict_class("where is your source code",model),intents)

1/1 [==============================] - 0s 41ms/step


[5.86497679e-20 5.60154874e-12 1.56286498e-10 9.38451811e-19
 1.05999227e-08 5.17447557e-15 2.13829394e-18 1.48113759e-12
 1.84389379e-11 8.87155373e-15 1.10396325e-13 6.36104494e-12
 1.92624866e-12 5.38796438e-12 3.52130174e-12 1.93711019e-12
 5.99858219e-11 2.32186801e-13 8.28273734e-12 1.31109509e-15
 5.79618897e-18 1.75468753e-14 4.77606939e-15 4.01133628e-14
 2.83500816e-13 1.00000000e+00 4.82485102e-12 4.20374157e-21]
maximum index = 25 probabiliy =  1.0


'My Source Code is Available in Github Repository https://github.com/PrathameshDhande22/Prakriti-Determine.'

## Creating the dataset

In [119]:
dataset={
}

In [120]:
for w in words:
    dataset[w]=[]
dataset["tag"]=[]
dataset

{"'m": [],
 "'re": [],
 "'s": [],
 'a': [],
 'about': [],
 'age': [],
 'annoying': [],
 'anyone': [],
 'are': [],
 'away': [],
 'bad': [],
 'be': [],
 'been': [],
 'best': [],
 'by': [],
 'bye': [],
 'ca': [],
 'call': [],
 'can': [],
 'chatbot': [],
 'chatgpt': [],
 'complete': [],
 'contact': [],
 'could': [],
 'created': [],
 'creator': [],
 'customer': [],
 'day': [],
 'despise': [],
 'detail': [],
 'determine': [],
 'diet': [],
 'disappointed': [],
 'do': [],
 'doing': [],
 'eat': [],
 'eating': [],
 'exercise': [],
 'exit': [],
 'fact': [],
 'feeling': [],
 'figure': [],
 'find': [],
 'food': [],
 'for': [],
 'fuck': [],
 'get': [],
 'give': [],
 'go': [],
 'good': [],
 'goodbye': [],
 'great': [],
 'hand': [],
 'hate': [],
 'have': [],
 'healthy': [],
 'hello': [],
 'help': [],
 'helpful': [],
 'hey': [],
 'hi': [],
 'hii': [],
 'hiii': [],
 'how': [],
 'i': [],
 'identify': [],
 'impressed': [],
 'improve': [],
 'in': [],
 'information': [],
 'irrelevant': [],
 'is': [],
 'job'

In [121]:
for train in training_data:
    for w,value in list(zip(words,train[0])):
        dataset[w].append(value)
    dataset["tag"].append(classes[np.argmax(train[1])])
dataset        

{"'m": [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 "'re": [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
 

In [122]:
for key in dataset.keys():
    print(key,dataset[key])

'm [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
're [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
's [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
a [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [131]:
index=[]

In [132]:
for docu in documents:
    index.append(" ".join(docu[0]))

In [133]:
index

['Hi',
 'How are you ?',
 'Hey there , how have you been ?',
 'Is anyone there ?',
 'Hello',
 'Good day',
 'Hiii',
 'Hii',
 'Bye',
 'See you there',
 'Goodbye',
 'Thanks',
 'Thank you',
 "That 's helpful",
 'Nice',
 'Good',
 'about you',
 'Who are you ?',
 'What are you ?',
 'Who you are ?',
 'what you can do',
 'What is the Purpose of the project',
 'what is your name',
 'what should I call you',
 'whats your name ?',
 'Could you help me ?',
 'give me a hand please',
 'Can you help ?',
 'What can you do for me ?',
 'I need a support',
 'I need a help',
 'support me please',
 'Complete the work',
 'My name',
 'user name',
 'you are not Good',
 'Nothing',
 'You are Bad',
 'Noting to worry',
 'Chatgpt is Best',
 'Irrelevant',
 'Can you help me find my prakriti',
 'what is my prakriti ?',
 'figure out my prakriti',
 'identify my prakriti',
 'Determine my Prakriti',
 'Determine',
 'Who are the creators',
 'who made these project',
 'Project Creators',
 'project is made by',
 'Project',
 'W

In [134]:
df=pd.DataFrame(dataset,index=index)
df

,'m,'re,'s,a,about,age,annoying,anyone,are,away,...,who,with,work,workout,worry,worthless,yeah,you,your,tag
Hi,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,greeting
How are you ?,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,greeting
"Hey there , how have you been ?",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,greeting
Is anyone there ?,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,greeting
Hello,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,greeting
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Give me nutrition facts,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,nutrition_information
I want to learn about healthy eating,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,nutrition_information
exit,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,exit
quit,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,exit


In [142]:
import os


df.to_csv("../dataset/sentence_dataset.csv")